In [10]:
import openeo
import numpy as np

url = "https://openeo-staging.dataspace.copernicus.eu"
connection = openeo.connect(url).authenticate_oidc()

Authenticated using refresh token.


In [9]:
datacube = connection.datacube_from_process("insar_preprocessing", 
                                 burst_id = 249435, 
                                 sub_swath = "IW2",
                                 temporal_extent = ["2024-08-09", "2024-09-02"],
                                 master_date = "2024-09-02",
                                 polarization = ["vv"],
                                ).save_result("GTiff")


job = datacube.create_job(title="sample_preprocessing_vv", job_options={
    "python-memory": "4200m",
})
job.start_and_wait()
job

0:00:00 Job 'j-25100212065049b5977728a74210ca7f': send 'start'
0:00:12 Job 'j-25100212065049b5977728a74210ca7f': created (progress 0%)
0:00:17 Job 'j-25100212065049b5977728a74210ca7f': created (progress 0%)
0:00:24 Job 'j-25100212065049b5977728a74210ca7f': queued (progress 0%)
0:00:31 Job 'j-25100212065049b5977728a74210ca7f': queued (progress 0%)
0:00:41 Job 'j-25100212065049b5977728a74210ca7f': queued (progress 0%)
0:00:54 Job 'j-25100212065049b5977728a74210ca7f': queued (progress 0%)
0:01:09 Job 'j-25100212065049b5977728a74210ca7f': queued (progress 0%)
0:01:29 Job 'j-25100212065049b5977728a74210ca7f': queued (progress 0%)
0:01:53 Job 'j-25100212065049b5977728a74210ca7f': queued (progress 0%)


KeyboardInterrupt: 

In [18]:
job = connection.job("j-25100212065049b5977728a74210ca7f")  # a job that has VV and VH bands
job

<BatchJob job_id='j-25100212065049b5977728a74210ca7f'>

Re use the pre-processed Sentinel-1 SLC data via STAC:

In [54]:
#cube = connection.load_stac_from_job("j-25100212065049b5977728a74210ca7f")
cube = connection.load_stac(
    url="https://openeo-staging.dataspace.copernicus.eu/openeo/1.2/jobs/j-25100212065049b5977728a74210ca7f/results/ODlhOWM4NjgtY2M0NC00NmJiLTkyMjktMTVmZjdmMjk4NWNm/c8d5e1a6a15cae61c630052fbaf523c1?expires=1760051374",
    temporal_extent=["2024-08-09", "2024-09-02"],
)
cube = cube.filter_bands(['i_VV', 'q_VV', 'grid_lat', 'grid_lon'])

bands_from_stac_collection: consulting items for band metadata


In [34]:
cube.metadata.band_names

['i_VV', 'q_VV', 'grid_lat', 'grid_lon']

In [55]:
cube.metadata.has_temporal_dimension()  # False, while should be True

False

In [56]:
cube.metadata = None  # clear to avoid temporal dimension issues

In [35]:
from openeo.processes import ProcessBuilder, array_element, array_create

def compute_parts_VV(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VV_mst")
    q_mst = array_element(x,label="q_VV_mst")
    i_slv = array_element(x,label="i_VV_slv")
    q_slv = array_element(x,label="q_VV_slv")
    return array_create([i_mst*i_slv + q_mst*q_slv,q_mst*i_slv - i_mst*q_slv,i_mst**2+q_mst**2,i_slv**2+q_slv**2])
    
def compute_parts_VH(x:ProcessBuilder):
    i_mst = array_element(x,label="i_VH_mst")
    q_mst = array_element(x,label="q_VH_mst")
    i_slv = array_element(x,label="i_VH_slv")
    q_slv = array_element(x,label="q_VH_slv")
    return array_create([i_mst*i_slv + q_mst*q_slv,q_mst*i_slv - i_mst*q_slv,i_mst**2+q_mst**2,i_slv**2+q_slv**2])
    
def compute_coherence_amplitude_VV(x:ProcessBuilder):
    q_num_VV = array_element(x,label="q_num_VV")
    i_num_VV = array_element(x,label="i_num_VV")
    mst_den = array_element(x,label="mst_den")
    slv_den = array_element(x,label="slv_den")
    
    coh_i = i_num_VV / ((mst_den * slv_den + 1e-12)**0.5)
    coh_q = q_num_VV / ((mst_den * slv_den + 1e-12)**0.5)
    
    return (coh_i**2+coh_q**2)**0.5

def compute_coherence_amplitude_VH(x:ProcessBuilder):
    q_num_VH = array_element(x,label="q_num_VH")
    i_num_VH = array_element(x,label="i_num_VH")
    mst_den = array_element(x,label="mst_den")
    slv_den = array_element(x,label="slv_den")
    
    coh_i = i_num_VH / ((mst_den * slv_den + 1e-12)**0.5)
    coh_q = q_num_VH / ((mst_den * slv_den + 1e-12)**0.5)
    
    return (coh_i**2+coh_q**2)**0.5

In [36]:
dates = [np.datetime64("2024-08-09"),np.datetime64("2024-08-21"),np.datetime64("2024-09-02")]
dates_pair = [[dates[0],dates[1]],[dates[1],dates[2]]]
# We use a loop over the dates we want to request and a combination of filter_temporal + merge_cubes to get only the dates we want:
for i,date_pair in enumerate(dates_pair):
    # The requested date until the day after, since openEO excludes the end date
    mst_date = date_pair[0]
    slv_date = date_pair[1]
    mst_temporal_extent = [str(mst_date),str(mst_date + np.timedelta64(1, "D"))]
    slv_temporal_extent = [str(slv_date),str(slv_date + np.timedelta64(1, "D"))]

    if i == 0:
        S1_slant_range_mst_bands = cube.filter_temporal(mst_temporal_extent)
        S1_slant_range_slv_bands = cube.filter_temporal(slv_temporal_extent)
        break # +++++++++++++++++++++ Do it only for one pair for testing
    else:
        S1_slant_range_mst_bands = cube.filter_temporal(mst_temporal_extent).merge_cubes(S1_slant_range_mst_bands)
        S1_slant_range_slv_bands = cube.filter_temporal(slv_temporal_extent).merge_cubes(S1_slant_range_slv_bands)
        

In [38]:
S1_slant_range_mst_bands = S1_slant_range_mst_bands.filter_bands([0,1]) #.drop_dimension("t") # "i_" real part,"q_" imaginary part
S1_slant_range_slv_bands = S1_slant_range_slv_bands.filter_bands([0,1]) #.drop_dimension("t")

In [39]:
S1_slant_range_mst_bands = S1_slant_range_mst_bands.rename_labels(dimension="bands",
                                                                  source=[0,1],
                                                                  target=["i_VV_mst","q_VV_mst"])
S1_slant_range_slv_bands = S1_slant_range_slv_bands.rename_labels(dimension="bands",
                                                                  source=[0,1],
                                                                  target=["i_VV_slv","q_VV_slv"])

In [40]:
S1_slant_range_mst_slv_bands = S1_slant_range_mst_bands.merge_cubes(S1_slant_range_slv_bands)

In [41]:
coh_numerator_denominator_VV = S1_slant_range_mst_slv_bands.apply_dimension(dimension="bands",
                                                               process=compute_parts_VV).rename_labels("bands",target=["i_num_VV","q_num_VV","mst_den","slv_den"])

In [42]:
coh_numerator_denominator_VV.execute_batch()

0:00:00 Job 'j-251002215256489d830b6a4d74971477': send 'start'
0:00:15 Job 'j-251002215256489d830b6a4d74971477': queued (progress 0%)
0:00:20 Job 'j-251002215256489d830b6a4d74971477': queued (progress 0%)
0:00:27 Job 'j-251002215256489d830b6a4d74971477': queued (progress 0%)
0:00:35 Job 'j-251002215256489d830b6a4d74971477': queued (progress 0%)
0:00:45 Job 'j-251002215256489d830b6a4d74971477': queued (progress 0%)
0:00:58 Job 'j-251002215256489d830b6a4d74971477': queued (progress 0%)
0:01:13 Job 'j-251002215256489d830b6a4d74971477': running (progress 10.3%)
0:01:32 Job 'j-251002215256489d830b6a4d74971477': running (progress 12.8%)
0:01:56 Job 'j-251002215256489d830b6a4d74971477': running (progress 15.8%)
0:02:27 Job 'j-251002215256489d830b6a4d74971477': running (progress 19.2%)
0:03:04 Job 'j-251002215256489d830b6a4d74971477': running (progress 23.1%)
0:03:51 Job 'j-251002215256489d830b6a4d74971477': running (progress 27.5%)
0:21:21 Job 'j-251002215256489d830b6a4d74971477': finished (p

<BatchJob job_id='j-251002215256489d830b6a4d74971477'>

In [43]:
from openeo.processes import inspect

In [49]:
cube

In [63]:
# Boxcar filter
range_dim   = 5
azimuth_dim = 1

# Create kernel
kernel = np.ones((range_dim,azimuth_dim))

# Pad with zeros to get a square kernel
pad = int((range_dim - azimuth_dim) / 2)
kernel = np.pad(kernel,((0,0),(pad,pad)))

coh_numerator_denominator_VV_boxcar = coh_numerator_denominator_VV.apply_kernel(kernel=kernel,
                                                                                factor=1/(range_dim*azimuth_dim),
                                                                                border=0)

coh_VV = coh_numerator_denominator_VV_boxcar.reduce_dimension(dimension="bands",
                                                              reducer=compute_coherence_amplitude_VV)

coordinate_bands = cube.filter_temporal(mst_temporal_extent).filter_bands([2,3])\
    .drop_dimension("t").rename_labels(dimension="bands", target=["latitude","longitude"])
coh_VV.metadata = None  # clear to avoid temporal dimension issues
coh_amplitude = coh_VV.add_dimension(name="bands",type="bands",label="coh_VV_amp").drop_dimension("t")\
    .merge_cubes(coordinate_bands).add_dimension(name="t", type="temporal", label="2020-05-01T00:00:00Z")

coh_amplitude = coh_amplitude.resample_spatial(projection="EPSG:32632",resolution=20.0,method="geocode").save_result(format="GTiff")

job_options = {
    #"image-name": "registry.stag.waw3-1.openeo-int.v1.dataspace.copernicus.eu/dev/openeo-geotrellis-kube-python311:20250903-133",
    "openeo-jar-path": "https://artifactory.vgt.vito.be/artifactory/libs-release-public/org/openeo/geotrellis-extensions/geocode/geotrellis-extensions-geocode.jar"
}
job = coh_amplitude.create_job(title="ClouDInSAR_coherence_from_preprocessing", job_options=job_options)

In [64]:
job.start_and_wait()

0:00:00 Job 'j-2510022346384d5db3e0b7aa93d2ed43': send 'start'
0:00:15 Job 'j-2510022346384d5db3e0b7aa93d2ed43': created (progress 0%)
0:00:20 Job 'j-2510022346384d5db3e0b7aa93d2ed43': created (progress 0%)
0:00:27 Job 'j-2510022346384d5db3e0b7aa93d2ed43': created (progress 0%)
0:00:35 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 4.8%)
0:00:45 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 6.3%)
0:00:57 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 8.1%)
0:01:13 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 10.3%)
0:01:32 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 12.8%)
0:01:56 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 15.7%)
0:02:26 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 19.1%)
0:03:04 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 23.0%)
0:03:51 Job 'j-2510022346384d5db3e0b7aa93d2ed43': running (progress 27.4%)
0:04:49 Job 'j-2510022346384d5db3e0b7aa93d2ed43':

JobFailedException: Batch job 'j-2510022346384d5db3e0b7aa93d2ed43' didn't finish successfully. Status: error (after 0:05:50).

In [12]:
job

<BatchJob job_id='j-2509040820244b149b936538719494b4'>

In [24]:
#job = connection.job("j-25060509012543ebb48aa5ee8c65b7ed")
job.get_results().download_files()

No assets found in job result metadata.


[PosixPath('/home/driesj/code/notebooks/s1-workflows/notebooks/job-results.json')]